In [0]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Dense, Flatten, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam

In [0]:
#image specs
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

In [0]:
# Size of noise vector used as input to Generator
z_dim = 100

#### Generator

In [0]:
def build_generator(img_shape, z_dim):
  model = Sequential()
  model.add(Dense(128, input_dim=z_dim))        #FC layer
  model.add(LeakyReLU(alpha=0.01))              #LeakyRelu activation
  model.add(Dense(28*28*1, activation='tanh'))  #op layr with tanh activation
  model.add(Reshape(img_shape))                 #Reshape generator op
  return model

#### Discriminator

In [0]:
def build_discriminator(img_shape):
  model = Sequential()
  model.add(Flatten(input_shape=img_shape))   #Flatten ip
  model.add(Dense(128))                       #FC layer
  model.add(LeakyReLU(alpha=0.01))            #LeakyRelu activation
  model.add(Dense(1, activation='sigmoid'))   #op layer for classification
  return model

#### Connect gen and dis

In [0]:
def build_gan(generator, discriminator):
  model = Sequential()
  model.add(generator)
  model.add(discriminator)
  return model